In [21]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, Activation, Add, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers
from keras.utils import to_categorical

img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(256, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    #Dense(128, activation='relu', input_shape=(2048, )),
    #BatchNormalization(),
    #Dropout(0.5),
    Dense(1),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

negated_encoded_b = Lambda(lambda x: -x, output_shape=(1,))(encoded_b)
diff = Add()([encoded_a, negated_encoded_b])

output = Activation("sigmoid")(diff)

model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

#optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_18 (InputLayer)            (None, 2048)          0                                            
____________________________________________________________________________________________________
sequential_9 (Sequential)        (None, 1)             525825      input_17[0][0]                   
                                                                   input_18[0][0]                   
____________________________________________________________________________________________________
input_17 (InputLayer)            (None, 2048)          0                                            
____________________________________________________________________________________________________
lambda_8 (Lambda)                (None, 1)             0           sequential_9[2][0]      

In [2]:
# data prepare
import cPickle as pickle
import numpy as np
import lmdb
import random

env = lmdb.open("./data/features")
txn = env.begin()

def get_XY(dataset):
    X1 = []
    X2 = []
    Y = []
    for img_a, s_a, img_b, s_b, cmpret in dataset:
        feature_a = txn.get(img_a)
        feature_b = txn.get(img_b)
        if feature_a is None or feature_b is None:
            continue
        feature_a = np.fromstring(feature_a, np.float32)
        feature_b = np.fromstring(feature_b, np.float32)
        X1.append(feature_a)
        X2.append(feature_b)
        Y.append(cmpret)
    X1 = np.array(X1)
    X2 = np.array(X2)
    Y = np.array(Y)
    return X1, X2, Y

batch_size = 128
train_list = pickle.load(open("./data/ranknet/train.list", 'rb'))
valid_list = pickle.load(open("./data/ranknet/valid.list", 'rb'))
print "# of trainset: %d"%len(train_list)
print "# of validset: %d"%len(valid_list)

X1_train, X2_train, Y_train = get_XY(train_list)
X1_valid, X2_valid, Y_valid = get_XY(valid_list)

train_list = []
valid_list = []

env.close()

# of trainset: 452243
# of validset: 4365


In [22]:
# training the model
#optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#model.compile(optimizer=optimizer,
#              loss='binary_crossentropy',
#              metrics=['accuracy'])

model.fit([X1_train, X2_train], Y_train, batch_size=batch_size, epochs=3, shuffle=True,
         validation_data=([X1_valid, X2_valid], Y_valid))

Train on 448305 samples, validate on 4303 samples
Epoch 1/3
448305/448305 [==============================] - 14s - loss: 0.6722 - acc: 0.6014 - val_loss: 0.6571 - val_acc: 0.6031
Epoch 2/3
448305/448305 [==============================] - 14s - loss: 0.6528 - acc: 0.6138 - val_loss: 0.6553 - val_acc: 0.6063
Epoch 3/3
448305/448305 [==============================] - 14s - loss: 0.6480 - acc: 0.6208 - val_loss: 0.6579 - val_acc: 0.6082


In [23]:
model.save_weights("./data/ranknet/ranknet.h5")

In [ ]:
# 输出valid的预估结果
import cPickle as pickle

model.load_weights("./data/ranknet/ranknet.h5")
Y_predict = model.predict([X1_valid, X2_valid], batch_size=batch_size)
assert(len(Y_predict) == len(valid_img_pairs))

Y_predict_map = {}
for i, (imga, imgb) in enumerate(valid_img_pairs):
    Y_predict_map[(imga, imgb)] = Y_predict[i, 1]
    
#pickle.dump(Y_predict_map, open("./data/valid_predict_by_rank_model.pick", 'wb'))